# Plotting the delivery areas
Using Geopandas 
Using shape file from: https://public.opendatasoft.com/explore/dataset/georef-netherlands-postcode-pc4/table/?refine.prov_name=Groningen&refine.gem_name=Groningen&location=10,53.21082,6.61855&basemap=jawg.light 

In [62]:
import pandas as pd
import geopandas as geopandas
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy

In [ ]:
geopandas.read_file('GEO-data/georef-netherlands-postcode-pc4.shp').head(5)

In [16]:
gdf_groningen = geopandas.read_file('GEO-data/georef-netherlands-postcode-pc4.shp')
gdf_groningen.explore()

In [70]:
# https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0
shp_zip_codes = geopandas.read_file("GEO-data/georef-netherlands-postcode-pc4.shp")[["pc4_code", "geometry"]]
#rename column
shp_zip_codes.columns = ["Postcode", "geometry"]
df_dabba = pd.read_excel("Dabba.xlsx", names = ["Klanteigenaar", "Aantal", "Adres", "Postcode", "Volgnummer"], usecols="B,F,G,H,R")
# change the 6 postal code to the first 4 numberecal codes
df_dabba["Postcode"] = df_dabba["Postcode"].apply(lambda x:x[:4])

# count the amount of unqiue postalcodes
df_dabba['Deliveries'] = df_dabba['Postcode'].map(df_dabba['Postcode'].value_counts())

In [ ]:
# Filter all the postalcodes in the geoframe based on the postalcodes in the dabba dataframe
shp_zip_codes_sorted_dabba = shp_zip_codes[shp_zip_codes["Postcode"].isin(df_dabba["Postcode"].unique())] 
# create area in km2 column
shp_zip_codes_sorted_dabba["Area (km2)"] = shp_zip_codes_sorted_dabba['geometry'].to_crs({'init': 'epsg:3395'}).map(lambda p: p.area / 10**6)


In [72]:
# merge dabba dataframe with the geoframe
merge = pd.merge(df_dabba[["Postcode", "Deliveries"]], shp_zip_codes_sorted_dabba, on="Postcode").drop_duplicates() 

# calculate the route length approximation per entry
k = 0.92
n = merge["Deliveries"]
A = merge["Area (km2)"]

# Calculate and add Beardwood as new column
merge["Beardwood approx"] = k * numpy.sqrt((n * A))
merge.head()

,Postcode,Deliveries,geometry,Area (km2),Beardwood approx
0,9726,481,"POLYGON ((6.55042 53.21567, 6.55138 53.21556, ...",1.670015,26.074773
481,9731,805,"POLYGON ((6.58247 53.24621, 6.58255 53.24588, ...",8.936459,78.031237
1286,9711,1531,"MULTIPOLYGON (((6.57755 53.21959, 6.57763 53.2...",2.269146,54.225889
2817,9724,470,"POLYGON ((6.58308 53.21218, 6.58426 53.21071, ...",1.151674,21.404326
3287,9717,629,"POLYGON ((6.55684 53.23095, 6.55720 53.23084, ...",1.698098,30.067308


In [78]:
# create a new geoframe based on the merge
results = geopandas.GeoDataFrame(merge[["Postcode", "Deliveries", "Area (km2)", "Beardwood approx"]], geometry=merge["geometry"])
#shape is 27
results.explore(
    column="Postcode",
    tooltip=['Postcode', 'Area (km2)', 'Deliveries', "Beardwood approx"],
    popup=True,
    tiles="CartoDB positron", 
    cmap="Set1", 
    style_kwds=dict(color="black"),
    name="Amount of deliveries in Groningen per postalcode area"
) 